In [166]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [69]:
def get_sibling(arr, tag, class_name):
    try:
        tag_label = arr.find(tag, class_=class_name)
        value = tag_label.find_next_sibling(tag).text.strip()
        return value
    except AttributeError:
        return "null"

In [70]:
def get_stars(arr, tag, class_name, tag2, class_name2):
    try:
        tag_label = arr.find(tag, class_=class_name)
        value = tag_label.find_next_sibling(tag)

        value2 = value.find_all(tag2, class_=class_name2)
        return len(value2)
    except AttributeError:
        return "null"

In [71]:
url = "https://www.airlinequality.com/airline-reviews/british-airways/page/"
page = 1
reviews_data = []

while True:
    response = requests.get(url+str(page))

    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.content, "html.parser")

    reviews = soup.find_all("div", class_="body")

    if not reviews:
        break

    for review in reviews:
        title = review.find("h2", class_="text_header").text.strip()
        subtitle = review.find("h3", class_="text_sub_header").text.strip()
        content = review.find("div", class_="text_content").text.strip()

        seat_type = get_sibling(review, "td", "review-rating-header cabin_flown")
        aircraft = get_sibling(review, "td", "review-rating-header aircraft")
        type_of_traveller = get_sibling(review, "td", "review-rating-header type_of_traveller")
        route = get_sibling(review, "td", "review-rating-header route")
        recommendation = get_sibling(review, "td", "review-rating-header recommended")
        date_flown = get_sibling(review, "td", "review-rating-header date_flown")


        seat_comfort = get_stars(review, "td", "review-rating-header seat_comfort", "span", "star fill")
        cabin_staff_service = get_stars(review, "td", "review-rating-header cabin_staff_service", "span", "star fill")
        food_and_beverage = get_stars(review, "td", "review-rating-header food_and_beverage", "span", "star fill")
        value_for_money = get_stars(review, "td", "review-rating-header value_for_money", "span", "star fill")
        inflight_entertainment = get_stars(review, "td", "review-rating-header inflight_entertainment", "span", "star fill")
        ground_service = get_stars(review, "td", "review-rating-header ground_service", "span", "star fill")
        wifi_and_connectivity = get_stars(review, "td", "review-rating-header wifi_and_connectivity", "span", "star fill")


        reviews_data.append(
            {
                "Title": title,
                "Subtitle": subtitle,
                "Content": content,
                "Seat Type": seat_type,
                "Aircraft": aircraft,
                "Type of Traveller": type_of_traveller,
                "Route": route,
                "Recommendation": recommendation,
                "Date Flown": date_flown,
                "Seat Comfort": seat_comfort,
                "Cabin Staff Service": cabin_staff_service,
                "Food and Beverage": food_and_beverage,
                "Value for Money": value_for_money,
                "Inflight Entertainment": inflight_entertainment,
                "Ground Service": ground_service,
                "Wifi and Connectivity": wifi_and_connectivity
            }
        )

    page+= 1

In [84]:
data = pd.DataFrame(reviews_data)


In [85]:
data.to_csv("british_airways_reviews.csv", index=False, encoding="utf-8-sig")

In [98]:
df = pd.read_csv("british_airways_reviews.csv", encoding="utf-8-sig")


In [202]:
df.head(1)

,Title,Subtitle,Content,Seat Type,Aircraft,Type of Traveller,Route,Recommendation,Date Flown,Seat Comfort,Cabin Staff Service,Food and Beverage,Value for Money,Inflight Entertainment,Ground Service,Wifi and Connectivity
0,“Horrendous Service”,R Heale (South Africa) 29th April 2025,✅ Trip Verified | I had an appalling experie...,Economy Class,A380,Solo Leisure,Johannesburg to London Heathrow,no,April 2025,1.0,1.0,NaN,1,NaN,1.0,NaN


### Data Cleaning and Transformation.
1. Remove the quotes in the title
2. Split the subtitle into reveiw_author, country, date posted
3. Split the content to trip_verified and main_review
4. Remove the emoji

In [238]:
clean_df = df.copy()

In [239]:
clean_df.isna().sum()

Title                        0
Subtitle                     0
Content                      0
Seat Type                    1
Aircraft                  1882
Type of Traveller          771
Route                      776
Recommendation               0
Date Flown                 778
Seat Comfort               127
Cabin Staff Service        142
Food and Beverage         3937
Value for Money              0
Inflight Entertainment    1251
Ground Service             850
Wifi and Connectivity     3243
dtype: int64

In [240]:
clean_df['Title'] = clean_df['Title'].replace(r'[\"“”‘’\']', "", regex=True)

In [241]:
# Extract the author, location, and date from the Subtitle column
clean_df['Author'] = df['Subtitle'].str.extract(r'^([A-Za-z0-9\s\'\-]+)(?=\s*\()')

clean_df['Location'] = clean_df['Subtitle'].str.extract(r'\((.*?)\)')

clean_df['Date Posted'] = clean_df['Subtitle'].str.extract(r'\)\s*(.*)$')

In [242]:
clean_df[['Trip Verified', 'Review']] = clean_df['Content'].str.split('|', expand=True)

In [243]:
clean_df['Trip Verified'] = clean_df['Trip Verified'].str.replace(r'^\W+', '', regex=True).str.strip()

In [244]:
clean_df.drop(columns=["Subtitle", "Content"], inplace=True)

In [245]:
clean_df = clean_df[["Title", "Author", "Location", "Date Posted", "Trip Verified", "Review", "Seat Type", "Aircraft", "Type of Traveller", "Route", "Recommendation", "Date Flown", "Seat Comfort", "Cabin Staff Service", "Food and Beverage", "Value for Money", "Inflight Entertainment", "Ground Service", "Wifi and Connectivity"]]

In [247]:
clean_df['Author'] = clean_df['Author'].str.replace(r'^\d+\s+reviews\s*', '', regex=True)
clean_df['Author'] = clean_df['Author'].str.replace(r'\s+', ' ', regex=True).str.strip()


In [250]:
clean_df.to_csv("clean_british_airways_reviews.csv", index=False, encoding="utf-8-sig")